# 0. Imports

In [1]:
import pandas as pd
import psycopg2 as pg

import credentials

# 1. Load Data

In [4]:
sql_query = """
select 
  u.id,
  u.gender,
  u.age,
  u.region_code,
  u.policy_sales_channel,
  i.previously_insured,
  i.annual_premium,
  i.vintage,
  v.driving_license,
  v.vehicle_age,
  v.vehicle_damage,
  i.response
from pa004.users u 
inner join pa004.insurance i on u.id = i.id 
inner join pa004.vehicle v on u.id = v.id
where u.id between 1 and 381109 -- limit id to guarantee the same data is fetched
"""

conn = pg.connect(
    user=credentials.USER,
    password=credentials.PWD,
    host=credentials.HOST,
    port=credentials.PORT,
    database=credentials.DB
)

df_raw = pd.read_sql(sql_query, conn)